In [ ]:
# this is the script i used to generate ground truths in form of graphs
# from binary masks
# i shall not try to clean it up since, these has been meant to be run just once
# and since the dataset is pretty unique and will not be released publically
# and the results seem ok for as much as they were verified (also in this script)

In [ ]:
import cv2
import IPython.display 
import importlib
import skimage.io as imgio
import numpy as np
import os
import torch
from graph_routines import graph_from_3D_mask, detectAndRemoveClusters, cycleBasis, reduceCycleBasis, minCycleBasis, verifyEdges
import scipy.sparse.csgraph as g
from skimage.morphology import skeletonize_3d
import pickle

def imshow(img):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)
    
def showCube(vol):
    v1=np.max(vol,axis=0)
    v2=np.max(vol,axis=1)
    v3=np.max(vol,axis=2)
    imshow(v1*255)
    imshow(v2*255)
    imshow(v3*255)
def show2Cubes(vol1,vol2):
    v11=np.max(vol1,axis=0)
    v12=np.max(vol1,axis=1)
    v13=np.max(vol1,axis=2)
    v21=np.max(vol2,axis=0)
    v22=np.max(vol2,axis=1)
    v23=np.max(vol2,axis=2)
    imshow(np.concatenate([v11,v21],1)*255)
    imshow(np.concatenate([v12,v22],1)*255)
    imshow(np.concatenate([v13,v23],1)*255)
    
def show3Cubes(vol1,vol2,vol3):
    v11=np.max(vol1,axis=0)
    v12=np.max(vol1,axis=1)
    v13=np.max(vol1,axis=2)
    v21=np.max(vol2,axis=0)
    v22=np.max(vol2,axis=1)
    v23=np.max(vol2,axis=2)
    v31=np.max(vol3,axis=0)
    v32=np.max(vol3,axis=1)
    v33=np.max(vol3,axis=2)
    imshow(np.concatenate([v11,v21,v31],1)*255)
    imshow(np.concatenate([v12,v22,v32],1)*255)
    imshow(np.concatenate([v13,v23,v33],1)*255)
    
def show4Cubes(vol1,vol2,vol3,vol4):
    v11=np.max(vol1,axis=0)
    v12=np.max(vol1,axis=1)
    v13=np.max(vol1,axis=2)
    v21=np.max(vol2,axis=0)
    v22=np.max(vol2,axis=1)
    v23=np.max(vol2,axis=2)
    v31=np.max(vol3,axis=0)
    v32=np.max(vol3,axis=1)
    v33=np.max(vol3,axis=2)
    v41=np.max(vol4,axis=0)
    v42=np.max(vol4,axis=1)
    v43=np.max(vol4,axis=2)
    imshow(np.concatenate([v11,v21,v31,v41],1)*255)
    imshow(np.concatenate([v12,v22,v32,v42],1)*255)
    imshow(np.concatenate([v13,v23,v33,v43],1)*255)

In [ ]:
from graph_routines import detectAndRemoveClusters, cycleBasis, reduceCycleBasis, minCycleBasis, verifyEdges

In [ ]:
imgdir="/cvlabdata2/home/kozinski/experimentsTorch/bbp_neurons/data_npy/img/train"
lbldir="/cvlabdata2/home/kozinski/experimentsTorch/bbp_neurons/newannot_npy/train"
testimgdir="/cvlabdata2/home/kozinski/experimentsTorch/bbp_neurons/data_npy/img/test"
testlbldir="/cvlabdata2/home/kozinski/experimentsTorch/bbp_neurons/newannot_npy/test"
exec(open("testFiles.txt").read())
exec(open("trainFiles.txt").read())


In [ ]:
datadir="/cvlabdata2/home/kozinski/experimentsTorch/bbp_neurons/data_npy/"
graphdir=os.path.join(datadir,"newlbl_graph")
#os.mkdir(graphdir)
traingraphdir=os.path.join(graphdir,"train")
#os.mkdir(traingraphdir)
testgraphdir=os.path.join(graphdir,"test")
#os.mkdir(testgraphdir)

In [ ]:

def removeCycle(edges,cycle):
    nodes_cycle=np.nonzero(cycle.reshape(edges.shape))[0]
    for k in nodes_cycle:
        if edges[k].sum()<3: # not connected to the outside of the cycle
            return k,True
    return 0,False

def removeCycles(edges,cycles):
    nodes= np.array(range(edges.shape[0]))
    nodes_to_be_removed=[]
    succ = True
    for c in cycles:
        print("attempting to remove cycle length",c.sum())
        n,res=removeCycle(edges,c)
        if res:
            print("succeeded")
        else:
            print("failed")
        succ=succ and res
        if res:
            nodes_to_be_removed.append(n)
    edges=np.delete(edges,nodes_to_be_removed,axis=0)
    edges=np.delete(edges,nodes_to_be_removed,axis=1)
    nodes=np.delete(nodes,nodes_to_be_removed,axis=0)
    return edges,nodes,succ
            

In [ ]:
# check for single dangling nodes
def checkDangling(edges):
    dangling=[]
    singlyConnected=np.nonzero(edges.sum(1)==1)[0]
    for k in singlyConnected:
        nb=np.argmax(edges[k]).item()
        if edges[nb].sum()>2:
            #print("dangling node")
            dangling.append(k)
    return np.array(dangling)

def removeDangling(edges,nodes):
    k=0
    while k<edges.shape[0]:
        if edges[k].sum()==1:
            # k is singly connected
            nb=np.argmax(edges[k]).item()
            if edges[nb].sum()>2:
                # the only neighbor of k is multiply connected
                # so k is a dangling node
                edges=np.delete(edges,(k),0)
                edges=np.delete(edges,(k),1)
                nodes=np.delete(nodes,(k),0)
                continue
        k=k+1
    return edges,nodes
        

In [ ]:
# prepare graphs for training ground truths
graphs=[]
for f in trainFiles:
  print(f)
  img=np.load(os.path.join(imgdir,f[1])).astype(np.float32)
  lbl=np.load(os.path.join(lbldir,f[1])).astype(np.float32)
  #img=np.copy(img[:,0:100,0:100,0:100])
  #lbl=np.copy(lbl[0:100,0:100,0:100])
  #lbl[lbl<2]=0
  #lbl=lbl/2
  lbl_skeletonized=skeletonize_3d(lbl)
  #showCube(lbl)
  inds,edges,d2=graph_from_3D_mask(lbl)
  #imshow(edges.astype(np.uint8)*255)
  edges_nc,nodes_nc=detectAndRemoveClusters(edges)
  inds_nc=inds[nodes_nc]
  #imshow(edges_nc.astype(np.uint8)*255)
  lbl_nc=np.zeros_like(lbl)
  for k in range(inds_nc.shape[0]):
    lbl_nc[inds_nc[k][0],inds_nc[k][1],inds_nc[k][2]]=1
  show2Cubes(lbl_nc,lbl)
  cycles_nc=minCycleBasis(edges_nc)
  if len(cycles_nc)>0:
    print("number cycles in the processed graph ",len(cycles_nc))
    edges_rc,nodes_rc,res=removeCycles(edges_nc,cycles_nc)
    inds_nc=inds_nc[nodes_rc]
    edges_nc=edges_rc
    if not res:
        print("failed to remove a cycle")
        break
  if not verifyEdges(edges_nc,inds_nc):
    print("connection between non-neighboring voxels")
    break
  num_cc_nc, component_labels_nc=g.connected_components(edges_nc)
  lbl_color_nc=np.zeros_like(lbl)
  for k in range(inds_nc.shape[0]):
    lbl_color_nc[inds_nc[k][0],inds_nc[k][1],inds_nc[k][2]]=component_labels_nc[k]+1
  lbl_color_nc=lbl_color_nc/float(num_cc_nc+1)
  #showCube(lbl_color_nc)
  #print( num_cc_nc )
  num_cc, component_labels=g.connected_components(edges)
  lbl_color=np.zeros_like(lbl)
  for k in range(edges.shape[0]):
    lbl_color[inds[k][0],inds[k][1],inds[k][2]]=component_labels[k]+1
  lbl_color=lbl_color/float(num_cc+1)
  show2Cubes(lbl_color,lbl_color_nc)
  #print( num_cc )
  if num_cc != num_cc_nc:
    print("different number of connected components")
    break
  assert(edges_nc.shape[0]==inds_nc.shape[0])
  graphs.append([f,edges_nc,inds_nc])

In [ ]:
# here you can check the graphs some more....

In [ ]:
# save training ground truth graphs
for gr in graphs:
    ff=open(os.path.join(traingraphdir,gr[0][0]),"bw")
    gt={'edges':gr[1],'node_coordinates':gr[2]}
    pickle.dump(gt,ff)
    ff.close()

In [ ]:
# prepare graphs for test ground truths
graphs=[]
for f in testFiles:
  print(f)
  img=np.load(os.path.join(testimgdir,f[1])).astype(np.float32)
  lbl=np.load(os.path.join(testlbldir,f[1])).astype(np.float32)
  #img=np.copy(img[:,0:100,0:100,0:100])
  #lbl=np.copy(lbl[0:100,0:100,0:100])
  #lbl[lbl<2]=0
  #lbl=lbl/2
  lbl_skeletonized=skeletonize_3d(lbl)
  #showCube(lbl)
  inds,edges,d2=graph_from_3D_mask(lbl)
  #imshow(edges.astype(np.uint8)*255)
  edges_nc,nodes_nc=detectAndRemoveClusters(edges)
  inds_nc=inds[nodes_nc]
  #imshow(edges_nc.astype(np.uint8)*255)
  lbl_nc=np.zeros_like(lbl)
  for k in range(inds_nc.shape[0]):
    lbl_nc[inds_nc[k][0],inds_nc[k][1],inds_nc[k][2]]=1
  show2Cubes(lbl_nc,lbl)
  cycles_nc=minCycleBasis(edges_nc)
  if len(cycles_nc)>0:
    print("number cycles in the processed graph ",len(cycles_nc))
    edges_rc,nodes_rc,res=removeCycles(edges_nc,cycles_nc)
    inds_nc=inds_nc[nodes_rc]
    edges_nc=edges_rc
    if not res:
        print("failed to remove a cycle")
        break
  if not verifyEdges(edges_nc,inds_nc):
    print("connection between non-neighboring voxels")
    break
  num_cc_nc, component_labels_nc=g.connected_components(edges_nc)
  lbl_color_nc=np.zeros_like(lbl)
  for k in range(inds_nc.shape[0]):
    lbl_color_nc[inds_nc[k][0],inds_nc[k][1],inds_nc[k][2]]=component_labels_nc[k]+1
  lbl_color_nc=lbl_color_nc/float(num_cc_nc+1)
  #showCube(lbl_color_nc)
  #print( num_cc_nc )
  num_cc, component_labels=g.connected_components(edges)
  lbl_color=np.zeros_like(lbl)
  for k in range(edges.shape[0]):
    lbl_color[inds[k][0],inds[k][1],inds[k][2]]=component_labels[k]+1
  lbl_color=lbl_color/float(num_cc+1)
  show2Cubes(lbl_color,lbl_color_nc)
  #print( num_cc )
  if num_cc != num_cc_nc:
    print("different number of connected components")
    break
  assert(edges_nc.shape[0]==inds_nc.shape[0])
  graphs.append([f,edges_nc,inds_nc])

In [ ]:
# check the graphs...

In [ ]:
# save the graphs for test ground truths
for gr in graphs:
    ff=open(os.path.join(testgraphdir,gr[0][0]),"bw")
    gt={'edges':gr[1],'node_coordinates':gr[2]}
    pickle.dump(gt,ff)
    ff.close()

In [ ]:
# read the graphs and remove dangling nodes

files=[trainFiles, testFiles]
lbldirs=[lbldir,testlbldir]
gdirs=[traingraphdir,testgraphdir]
for fls,lds,gd in zip(files,lbldirs,gdirs):
    print(lds)
    for f in fls:
        print(f)
        ff=open(os.path.join(gd,f[1]),"rb")
        gt=pickle.load(ff)
        ff.close()
        nc=gt['node_coordinates']
        edges_graph=gt['edges']
        dangling=checkDangling(edges_graph)
        print("before removal, there were {} dangling nodes".format(len(dangling)))
        edges_graph,nc=removeDangling(edges_graph,nc)
        dangling=checkDangling(edges_graph)
        print("after removal, there were {} dangling nodes".format(len(dangling)))
        ff=open(os.path.join(gd,f[1]),"bw")
        gt={'edges':edges_graph,'node_coordinates':nc}
        pickle.dump(gt,ff)
        ff.close()


In [ ]:
# verify the saved data: both test and train graphs
def colorComponents(lbl,edges,coords):
    lbl_=np.zeros_like(lbl)
    num_cc, component_labels=g.connected_components(edges)
    for k in range(coords.shape[0]):
        lbl_[coords[k][0],coords[k][1],coords[k][2]]=component_labels[k]+1
    lbl_=lbl_/float(num_cc+1)
    return lbl_,num_cc

files=[trainFiles, testFiles]
lbldirs=[lbldir,testlbldir]
gdirs=[traingraphdir,testgraphdir]
for fls,lds,gd in zip(files,lbldirs,gdirs):
    print(lds)
    for f in fls:
        print(f)
        lbl=np.load(os.path.join(lds,f[1])).astype(np.float32)
        lbl[lbl<2]=0
        lbl=lbl/2
        inds,edges,d2=graph_from_3D_mask(lbl)
        ff=open(os.path.join(gd,f[1]),"rb")
        gt=pickle.load(ff)
        ff.close()
        nc=gt['node_coordinates']
        edges_graph=gt['edges']
        lbl_graph,n_graph=colorComponents(lbl,edges_graph,nc)
        lbl_color,n_orig=colorComponents(lbl,edges,inds)
        show3Cubes(lbl_color>0,(lbl_graph>0)!=(lbl_color>0),lbl_graph>0)
        assert(n_graph==n_orig)
